### nn.Module是PyTorch提供的神经网络类，并在类中实现了网络各层的定义及前向计算与反向传播机制。在实际使用时，如果想要实现某个神经网络，只需继承nn.Module，在初始化中定义模型结构与参数，在函数forward()中编写网络前向过程即可。

下面具体以一个由两个全连接层组成的感知机为例，介绍如何使用nn.Module构造模块化的神经网络。新建一个**perception.py**文件。

In [1]:
import torch

from perception import Perception

In [2]:
#实例化一个网络，并赋值全连接中的维数，最终输出二维代表了二分类
perception = Perception(2, 3, 2)
perception

Perception(
  (layer1): Linear()
  (layer2): Linear()
)

In [3]:
#named_parameters()可以返回学习参数的迭代器，分别为参数名与参数值
for name, parameter in perception.named_parameters():
    print(name, parameter)

layer1.w Parameter containing:
tensor([[-1.1493, -1.2095,  0.6069],
        [-0.0972,  0.7227,  0.2993]], requires_grad=True)
layer1.b Parameter containing:
tensor([-0.7699,  1.5825, -0.9299], requires_grad=True)
layer2.w Parameter containing:
tensor([[0.8139, 0.4987],
        [0.7059, 0.4061],
        [0.4427, 0.7548]], requires_grad=True)
layer2.b Parameter containing:
tensor([0.7511, 0.5498], requires_grad=True)


In [4]:
#随机生成数据，注意这里的4代表了样本数为4，每个样本有两维
data = torch.randn(4, 2)
data

tensor([[-1.3074, -0.1202],
        [-1.1213, -0.7599],
        [-1.2946, -1.2666],
        [-0.2066, -0.7217]])

In [5]:
#将输入数据传入perception，perception()相当于调用perception中的forward()函数
output = perception(data)
output

tensor([[0.8852, 0.8001],
        [0.8790, 0.7936],
        [0.8814, 0.7938],
        [0.8479, 0.7734]], grad_fn=<SigmoidBackward0>)

## 实际使用nn.Module搭建神经网络时，应注意以下5点：

### 1.nn.Parameter函数

In [6]:
#在类的__init__()中需要定义网络学习的参数，在此使用nn.Parameter()函数定义了全连接中的ω和b，
#这是一种特殊的Tensor的构造方法，**默认需要求导，即requires_grad为True**。

### 2.forward()函数与反向传播

In [7]:
#forward()函数用来进行网络的前向传播，并需要传入相应的Tensor，例如上例的perception(data)即是直接调用了forward()。
#在具体底层实现中，perception.__call__(data)将类的实例perception变成了可调用对象perception(data)，
#而在perception.__call__(data)中主要调用了forward()函数，具体可参考官方代码。
#nn.Module可以自动利用Autograd机制实现反向传播，不需要自己手动实现。

### 3.多个Module的嵌套
在Module的搭建时，可以嵌套包含子Module，上例的Perception中调用了Linear这个类，这样的代码分布可以使网络更加模块化，提升代码的复用性。在实际的应用中，PyTorch也提供了绝大多数的网络层，如全连接、卷积网络中的卷积、池化等，并自动实现前向与反向传播。

### 4.nn.Module与nn.functional库
在PyTorch中，还有一个库为nn.functional，同样也提供了很多网络层与函数功能，但与nn.Module不同的是，利用nn.functional定义的网络层不可自动学习参数，还需要使用nn.Parameter封装。nn.functional的设计初衷是对于一些不需要学习参数的层，如激活层、BN（Batch Normalization）层，可以使用nn.functional，这样这些层就不需要在nn.Module中定义了。
总体来看，对于需要学习参数的层，最好使用nn.Module，对于无参数学习的层，可以使用nn.functional，当然这两者间并没有严格的好坏之分。

### 5.nn.Sequential()模块
当模型中只是简单的前馈网络时，即上一层的输出直接作为下一层的输入，这时可以采用nn.Sequential()模块来快速搭建模型，而不必手动在forward()函数中一层一层地前向传播。因此，如果想快速搭建模型而不考虑中间过程的话，推荐使用nn.Sequential()模块。

在上面的例子中，Perception类中的layer1与layer2是直接传递的，因此该Perception类可以使用nn.Sequential()快速搭建。在此新建一个**perception_sequential.py**文件，在终端中进入上述perception_sequential.py文件的同级目录下，输入python3进入交互环境，使用如下指令即可调用该网络结构。

In [8]:
import torch

from perception_sequential import Perception

In [9]:
#构建类的实例，并表明在CUDA上
model = Perception(100, 1000, 10).cuda()
#打印model结构，会显示Sequential中每一层的具体参数配置
model

Perception(
  (layer): Sequential(
    (0): Linear(in_features=100, out_features=1000, bias=True)
    (1): Sigmoid()
    (2): Linear(in_features=1000, out_features=10, bias=True)
    (3): Sigmoid()
  )
)

In [10]:
input = torch.randn(100).cuda()
input

tensor([-0.3826,  0.3045, -0.9604, -1.4223,  0.7467,  0.4634,  1.2368, -0.6277,
         0.5601,  1.0362,  2.1163,  0.1522, -1.1923,  1.2737, -1.0812, -1.4766,
        -0.7104, -0.3865,  0.9446, -0.6162, -1.1548, -1.4467,  1.2442, -0.9931,
        -1.2798, -1.1044, -0.6918, -0.8467,  0.8229, -0.5382,  0.0155,  0.0901,
        -2.1605,  0.9554, -0.7610,  0.2805,  2.3617,  0.8847,  1.6083,  1.4039,
         1.0145, -0.1967,  0.8703, -1.1285, -1.8560, -1.6603,  0.7273,  1.6222,
        -0.5677, -0.3985, -0.9211, -0.2390,  1.2600,  0.3650,  3.5262,  0.0681,
         1.1790,  1.2994, -0.4821, -1.6063, -1.8894, -0.2753, -0.9812, -1.9853,
         0.6556,  2.2316,  1.5557,  1.3690, -1.5726, -1.8738,  0.4283,  0.6398,
         0.8847, -0.4285, -0.3912,  0.8586, -0.4006, -1.2287, -2.4141, -1.2865,
         0.6264,  0.9491,  0.2058, -0.0556, -1.2103, -1.9216,  0.7622,  0.2301,
        -2.0514, -0.8415,  0.7683, -0.2362,  0.3185,  0.4018,  0.1568,  1.0230,
         0.7887,  0.0948, -0.0041, -0.30

In [11]:
#将输入传入实例化的模型
output = model(input)
output

tensor([0.4986, 0.5077, 0.5177, 0.6745, 0.3798, 0.5611, 0.5760, 0.4005, 0.5632,
        0.5139], device='cuda:0', grad_fn=<SigmoidBackward0>)

In [12]:
output.shape

torch.Size([10])